In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
from google.colab import files

# Load datasets
# Upload the Excel file
uploaded = files.upload()


Saving Dataset_clean_2.xlsx to Dataset_clean_2.xlsx


In [41]:
def preprocess_sequences(sequences, sst3_labels, encoder, decoder, maxlen=300):
    X = pad_sequences(encoder.texts_to_sequences(seq2kmers(sequences, n=3)), maxlen=maxlen, padding='post')
    y = to_categorical(pad_sequences(decoder.texts_to_sequences(sst3_labels), maxlen=maxlen, padding='post'))
    return X, y
# Get the file name
file_name = next(iter(uploaded))

# Read the Excel file into a DataFrame
df = pd.read_excel(pd.ExcelFile(file_name))


In [51]:
ss_all = df.query('len.between(100, 300) & ~has_nonstd_aa')
sequences, sst3_labels = df['seq'].tolist(), df['sst3'].tolist()


def seq2kmers(sequences, n):
    return [sequence[i:i+n] for sequence in sequences for i in range(len(sequence)-n+1)]

In [52]:
# Train-validation-test split
train_indices, test_indices = train_test_split(range(len(sequences)), test_size=0.2, random_state=42)
train_indices, valid_indices = train_test_split(train_indices, test_size=0.25, random_state=42)

train_seqs, train_sst3 = [sequences[i] for i in train_indices], [sst3_labels[i] for i in train_indices]
valid_seqs, valid_sst3 = [sequences[i] for i in valid_indices], [sst3_labels[i] for i in valid_indices]
test_seqs, test_sst3 = [sequences[i] for i in test_indices], [sst3_labels[i] for i in test_indices]

# Tokenization and Padding

In [53]:
# Tokenization and Padding
encoder = Tokenizer()
encoder.fit_on_texts(seq2kmers(train_seqs, n=3))
decoder = Tokenizer(char_level=True)
decoder.fit_on_texts(train_sst3)

In [54]:
# Train-validation-test split for sequences and labels
X_train, y_train = train_seqs, train_sst3
X_valid, y_valid = valid_seqs, valid_sst3
X_test, y_test = test_seqs, test_sst3



In [55]:
# Build and Compile Model
def q3_acc(y_true, y_pred):
    y, y_ = tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1)
    mask = tf.greater(y, 0)
    return K.cast(K.equal(tf.boolean_mask(y, mask), tf.boolean_mask(y_, mask)), K.floatx())


model = Sequential([
    Embedding(input_dim=len(encoder.word_index) + 1, output_dim=128, input_length=300),
    Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1)),
    TimeDistributed(Dense(len(decoder.word_index) + 1, activation='softmax'))
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy", q3_acc])



In [56]:
# Train the model
X_train, y_train = preprocess_sequences(X_train, y_train, encoder, decoder)
X_valid, y_valid = preprocess_sequences(X_valid, y_valid, encoder, decoder)
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid, y_valid), verbose=1)


ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt

# Encode the test set
X_test = pad_sequences(encoder.texts_to_sequences(seq2kmers(test_seqs, n=3)), maxlen=300, padding='post')
y_test = to_categorical(pad_sequences(decoder.texts_to_sequences(test_sst3), maxlen=300, padding='post'))

# Test set predictions and Q3 Accuracy
test_preds = model.predict(X_test)

q3 = np.sum([np.sum(q3_acc(y_test[i], pred)) for i, pred in enumerate(test_preds)])
total_count = np.sum([len(q3_acc(y_test[i], pred)) for pred in test_preds])
q3_accuracy = np.round(q3 / total_count, 2)

print('Test Set Q3 Accuracy:', q3_accuracy)

# Plot accuracy graph
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_valid, y_valid), verbose=1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()